# Script Writer Workflow Using CrewAI


In this notebook, we demonstrate how to build and collaborating agentic AI system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather current information from the internet about given topic.
2. Make a conversational script out of the information.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create two agents for data gathering and script creation.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [ ]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai==0.95.0
!pip install crewai-tools==0.25.8
!pip install duckduckgo-search
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.6 MB/s eta 0:0

In [ ]:
from crewai import Agent, Task
from crewai.tools import tool
from crewai.llm import LLM
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime
import os

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)


### Step 2: Define the Agents

In [ ]:
# Alternative Search tool from Crew AI tools. Try this and see the difference
from google.colab import userdata
from crewai_tools import EXASearchTool
import os

os.environ['EXA_API_KEY'] = userdata.get('exa_api_key')

Exa_Search_Tool = EXASearchTool ()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1089: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:35: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see t

In [ ]:
from crewai import Agent
#from langchain_community.tools.exa.tool import ExaSearchTool

## Agent designed to gather information about a topic from internet.
content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    # llm = 'gemini/gemini-2.0-flash-lite',
    # llm = 'groq/gemma2-9b-it',
    llm = 'gpt-4o',
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information. Consider you are on : '+Today),
    tools = [Exa_Search_Tool],
    cache = True,
    max_iter = 5
)

## Agent to create a script out of information.
## Back story sets the context for the agent.
# script_llm = LLM (model='groq/llama3-70b-8192', temperature=1.2)
script_llm = LLM (model='gpt-4o', temperature=0.9)
Script_Writer = Agent (
    role = 'Script Writer',
    goal = 'With the details given to you create an interesting conversational script out of it',
    llm=script_llm,
    verbose = True,
    backstory = ('You are an expert in Shakesperean literature and dialogue. You are very good in creating conversations with the given chain of information.\
                 Tell as a script in 200 words.\
                 Consider you are on '+Today),
)

os.environ['OTEL_SDK_DISABLED'] = "True"


### Step 3: Define the Tasks

In [ ]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to create script.
create_a_script = Task (
                      description = "Considering the given details in time order make an interesting conversation",
                      expected_output = "A humorous conversation connecting key details",
                      agent = Script_Writer,
                      context = [get_details],
                      output_file = 'Script.txt'
                    )



### Step 4: Set Up the Crew

In [ ]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, Script_Writer],
    tasks = [get_details, create_a_script],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

### Step 5: Run the Crew and Observe Results

In [ ]:
# Set your OpenAI API key or any other LLM API key
from google.colab import userdata

# Set your Groq API key or any other LLM API key
# os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('openai_key')
# os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Run the crew with a specific stock
result = crew.kickoff (inputs={'topic' : "Musk Vs Trump"})


# Agent: content explorer
## Task: Get latest, trending, interesting information and news about Musk Vs Trump
2025-06-08 09:25:26.310493


# Agent: content explorer
## Thought: To find the latest and trending information about the topic "Musk Vs Trump," I need to search the internet for recent news articles, updates, and any interesting trivia related to these two figures.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Musk Vs Trump latest news June 2025\"}"
## Tool Output: 

Search results: Title: Trump and Musk feud draws reactions from billionaires, politicians and pundits. Here's who said what
Score: 0.4283819794654846
Url: https://www.cnbc.com/2025/06/06/trump-musk-feud-draws-reactions-from-billionaires-politicians-and-pundits.html
ID: https://www.cnbc.com/2025/06/06/trump-musk-feud-draws-reactions-from-billionaires-politicians-and-pundits.html
---
Title: Threats of contract breach, impeachment as Musk, Trump clash
Score: 0.42270413041114807
Url: https://


---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
